In [1]:

from google.colab import userdata
google_api_key=userdata.get('google_api_key')


In [2]:
import google.generativeai as genai
genai.configure(api_key=google_api_key)

In [4]:
MODEL_CONFIG = {
  "temperature": 0.2}

In [3]:

model = genai.GenerativeModel(model_name = "gemini-1.0-pro",
                              generation_config = MODEL_CONFIG)

In [4]:
!pip install langchain

In [5]:
!pip install langchain-google-genai

In [14]:
! pip install -q --upgrade google-generativeai langchain-google-genai

**Gemini API + Langchain**

In [3]:
from IPython.display import display
from IPython.display import Markdown

import os

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [4]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)


result = llm.invoke("What is a LLM?")

Markdown(result.content)


LLM stands for **Large Language Model**.

LLMs are a type of artificial intelligence (AI) that are trained on massive datasets of text and code. They are designed to understand and generate human-like text, and to perform a variety of language-related tasks, such as:

* **Natural language understanding (NLU)**: Understanding the meaning of text.
* **Natural language generation (NLG)**: Generating text from data or instructions.
* **Machine translation**: Translating text from one language to another.
* **Text summarization**: Summarizing long pieces of text into shorter ones.
* **Question answering**: Answering questions based on a given context.
* **Code generation**: Generating code from natural language instructions.

LLMs are typically trained on billions or even trillions of words of text, and they use a variety of deep learning techniques to learn the patterns and relationships in language. This allows them to generate text that is both coherent and fluent, and to perform language-related tasks with a high degree of accuracy.

Some of the most well-known LLMs include:

* **GPT-3** from OpenAI
* **BLOOM** from Google AI
* **Jurassic-1** from AI21 Labs
* **Megatron-Turing NLG** from Microsoft and NVIDIA

LLMs are still under development, but they have already shown great promise for a wide range of applications, including:

* **Chatbots and virtual assistants**: Providing natural language-based customer support and information.
* **Content creation**: Generating marketing copy, news articles, and other written content.
* **Language learning**: Helping people to learn new languages.
* **Education**: Providing personalized learning experiences and answering student questions.
* **Healthcare**: Assisting doctors with diagnosis and treatment planning.

As LLMs continue to improve, they are likely to have an even greater impact on our lives. They have the potential to revolutionize the way we communicate, learn, and work.

In [7]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest one fency name for this."
)

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
response=chain.run("Mexican")
print(response)

- Hacienda de los Sabores


**Sequential Chain**

In [10]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest one fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
print(chain({"cuisine": "indian"}))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'indian', 'restaurant_name': 'Tandoori Royale', 'menu_items': '**Appetizers**\n\n* **Tandoori Chicken Tikka:** Boneless chicken marinated in yogurt, spices, and herbs, grilled in a tandoor\n* **Vegetable Samosas:** Crispy pastries filled with a savory vegetable mixture of potatoes, peas, and spices\n* **Onion Bhajis:** Fried onion fritters seasoned with cumin and coriander\n* **Papadum:** Thin, crispy lentil crackers served with chutney\n* **Mango Lassi:** A refreshing yogurt-based drink blended with mango pulp\n\n**Main Courses**\n\n* **Tandoori Mixed Grill:** A platter of tandoori-grilled meats including chicken, lamb, and shrimp\n* **Butter Chicken:** Creamy tomato-based curry with boneless chicken\n* **Palak Paneer:** Spinach-based curry with cubes of paneer cheese\n* **Lamb Vindaloo:** Spicy Goan curry with tender lamb\n* **Vegetable Biryani:** Aromatic rice dish with vegetables, spices, and herbs\n* **Rogan Josh:** Kashmiri lamb curry with aromatic spices\n\n**Sides**

**ConversationBufferMemory**

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

El Palacio de los Sabores Exquisitos


In [15]:
name = chain.run("Arabic")
print(name)

Al-Majlis al-Maliki


In [16]:
print(chain.memory.buffer)

Human: Mexican
AI: El Palacio de los Sabores Exquisitos
Human: Arabic
AI: Al-Majlis al-Maliki


**ConversationChain**

In [17]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [18]:
convo.run("Who won the first cricket world cup?")

'West Indies'

In [19]:
convo.run("Who was the captain ofthe winning team?") #wrong answer

'Frank Worrell'

In [20]:
convo.run("How much is 5+5?")

'10'

In [23]:
convo.run("Who was the captain ofthe first cricket world cup?")

'Frank Worrell'

In [24]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI: West Indies
Human: Who was the captain ofthe winning team?
AI: Frank Worrell
Human: How much is 5+5?
AI: 10
Human: Who was the captain ofthe first world cup?
AI: Frank Worrell
Human: Who was the captain ofthe first cricket world cup?
AI: Frank Worrell


**ConversationBufferWindowMemory**

In [25]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=llm,
    memory=memory
)
convo.run("Who was the captain of first cricket world cup?")

'**AI:** Clive Lloyd'

**Without Langchain**

In [ ]:
local_llm=model

In [7]:
local_llm

 genai.GenerativeModel(
   model_name='models/gemini-1.0-pro',
   generation_config={'temperature': 0.2}.
   safety_settings={}
)

In [12]:
name = local_llm.generate_content("I want to open a restaurant for Chinese food. Suggest one fency name for this.")


In [13]:
print(name.text)

Celestial Dynasty


Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
LLM: The language model powering the agent.
Agent: The agent to use.

In [ ]:
#https://github.com/RGGH/OpenAI_SQL/blob/master/LangChain_01.ipynb